# FASTVIT PET MOBILE

## 1. Environment setup

In [1]:
!git clone https://github.com/HenryNVP/fastvit-pet-mobile.git
%cd fastvit-pet-mobile

Cloning into 'fastvit-pet-mobile'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 63 (delta 11), reused 57 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (63/63), 3.12 MiB | 37.63 MiB/s, done.
Resolving deltas: 100% (11/11), done.
/content/fastvit-pet-mobile


In [2]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 138.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.3 MB/s eta 0:00:00


## 2. Download and prepare data

This downloads the official dataset and prepares 256x256 splits.

In [3]:
!python scripts/get_data.py
!python scripts/split_dataset.py

/content/fastvit-pet-mobile/scripts/get_data.py:33: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tf.extractall(destination)
Prepared train=3680 val=1832 test=1837


## 3. Finetune teacher model

In [5]:
import torch
import fastvit.models
from timm.models import create_model
from fastvit.models.modules.mobileone import reparameterize_model

# Create the model
teacher_model = create_model("fastvit_t8", pretrained=False)

# Download and load checkpoint
!wget -O fastvit_t8.pth.tar \
"https://docs-assets.developer.apple.com/ml-research/models/fastvit/image_classification_models/fastvit_t8.pth.tar"

checkpoint = torch.load("fastvit_t8.pth.tar", map_location='cpu')
teacher_model.load_state_dict(checkpoint['state_dict'])

# ... train ...

# For inference
teacher_model.eval()
model_inf = reparameterize_model(teacher_model)
# Use model_inf at test-time

--2025-11-19 22:45:05--  https://docs-assets.developer.apple.com/ml-research/models/fastvit/image_classification_models/fastvit_t8.pth.tar
Resolving docs-assets.developer.apple.com (docs-assets.developer.apple.com)... 17.253.118.202, 2403:300:a32:f100::2
Connecting to docs-assets.developer.apple.com (docs-assets.developer.apple.com)|17.253.118.202|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16378213 (16M) [application/x-tar]
Saving to: ‘fastvit_t8.pth.tar’

fastvit_t8.pth.tar  100%[===================>]  15.62M  11.1MB/s    in 1.4s    

2025-11-19 22:45:07 (11.1 MB/s) - ‘fastvit_t8.pth.tar’ saved [16378213/16378213]



In [6]:
from PIL import Image
from torchvision import transforms

# Load image
img = Image.open("/content/fastvit-pet-mobile/data/train/Abyssinian/Abyssinian_1.jpg").convert("RGB")

# Preprocess
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

input_tensor = preprocess(img).unsqueeze(0)  # add batch dimension

with torch.no_grad():
    output = model_inf(input_tensor)  # shape: [1, num_classes]
    predicted_class = torch.argmax(output, dim=1)
    print("Predicted class index:", predicted_class.item())

Predicted class index: 273


In [9]:
!python fastvit/train.py data --model fastvit_t8 -b 128 --lr 1e-3 --native-amp --mixup 0.2 --output ./output/teacher --input-size 3 256 256


/usr/local/lib/python3.12/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.12/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
If for semantic segmentation, please install mmsegmentation first
If for detection, please install mmdetection first
/content/fastvit-pet-mobile/fastvit/models/fastvit.py:956: UserWarning: Overwriting fastvit_t8 in registry with fastvit.models.fastvit.fastvit_t8. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  @register_model
/content/fastvit-pet-mobile/fastvit/models/fastvit.py:978: Us

In [10]:
print(teacher_model)

FastViT(
  (patch_embed): Sequential(
    (0): MobileOneBlock(
      (se): Identity()
      (activation): GELU(approximate='none')
      (rbr_conv): ModuleList(
        (0): Sequential(
          (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (rbr_scale): Sequential(
        (conv): Conv2d(3, 48, kernel_size=(1, 1), stride=(2, 2), bias=False)
        (bn): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): MobileOneBlock(
      (se): Identity()
      (activation): GELU(approximate='none')
      (rbr_conv): ModuleList(
        (0): Sequential(
          (conv): Conv2d(48, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=48, bias=False)
          (bn): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (rbr_scale): S

In [12]:
tuned_teacher = create_model(
    "fastvit_t8",
    pretrained=False,
    num_classes=37,
)

checkpoint = torch.load("/content/fastvit-pet-mobile/output/teacher/20251119-225323-fastvit_t8-256/model_best.pth.tar", map_location='cpu')
tuned_teacher_model.load_state_dict(checkpoint['state_dict'])


UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL argparse.Namespace was not an allowed global by default. Please use `torch.serialization.add_safe_globals([argparse.Namespace])` or the `torch.serialization.safe_globals([argparse.Namespace])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about types accepted by defa

## 4. Setup and finetune student model

## 5. Model evaluation

## 6. Export model

## 7. Summary and conclusion